In [1]:
from sklearn.model_selection import KFold

In [2]:
class KFoldCrossValidation(object):
    def __init__(self, arff_path, class_attribute_index=0):
        self.attributes, self.sample_data, empty = ARFFLoader(arff_path).load_attributes_and_samples(class_attribute_index, 100)
        self.class_attribute_name = self.attributes[0]['name']

    def perform_k_fold_cross_validation(self, n_splits, classifier):
        self.sample_data = np.array(self.sample_data)
        
        scores = []
        fold_split = list(KFold(n_splits).split(np.array(self.sample_data)))
        
        for progress_iterator in log_progress(range(0, n_splits), name="Walidacja", every=1):
            train_index, test_index  = fold_split[progress_iterator]
            
            classifier.sample_data = self.sample_data[train_index]
            classifier.attributes = self.attributes
            network = classifier.find_optimal_net()
            
            correct_counter = 0
            incorrect_counter = 0
            
            for element in self.sample_data[test_index]:
                result = From(network, self.sample_data[test_index]).given_many(element).classify(self.class_attribute_name)
                if self.check_winner(result) == element[self.class_attribute_name]:
                    correct_counter += 1
                else: 
                    incorrect_counter += 1
                
            total = correct_counter + incorrect_counter 
            scores.append(correct_counter / total)
            
        return np.array(scores)
    
    def check_winner(self, result_table):
        biggest_result = -99999
        biggest_result_index = 0

        for index, result in enumerate(result_table):
            if result['value'] > biggest_result:
                biggest_result = result['value']
                biggest_result_index = index

        return result_table[biggest_result_index]['state']